In [67]:
import numpy as np
from ipynb.fs.full.preprocessing import *
from scipy.stats import multivariate_normal
from scipy.stats import norm
from sklearn.decomposition import PCA
# from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import IsolationForest

Performance metrics...

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def get_performance(y_true, y_pred):
    # make sure anomalies are 1
    print("Test Accuracy:", accuracy_score(y_true, y_pred))
    print("Test Precision:", precision_score(y_true, y_pred))
    print("Test Recall:", recall_score(y_true, y_pred))
    print("Test F1 Score:", f1_score(y_true, y_pred))

Reading data...

In [95]:
PFA = 0.05
# traindata, testdata, testlabel = read_skab_data()
traindata, testdata, testlabel = read_smd_data()
traindata

[array([[1.01010e-02, 1.64200e-03, 1.99600e-03, ..., 1.41966e-01,
         0.00000e+00, 0.00000e+00],
        [1.01010e-02, 6.26000e-04, 1.55300e-03, ..., 1.49766e-01,
         0.00000e+00, 0.00000e+00],
        [1.01010e-02, 2.35000e-04, 1.10900e-03, ..., 1.35725e-01,
         0.00000e+00, 0.00000e+00],
        ...,
        [1.01010e-02, 1.09500e-03, 1.88600e-03, ..., 1.23245e-01,
         0.00000e+00, 0.00000e+00],
        [2.02020e-02, 1.32900e-03, 1.88600e-03, ..., 2.48050e-01,
         0.00000e+00, 0.00000e+00],
        [1.01010e-02, 8.60000e-04, 1.55300e-03, ..., 1.29485e-01,
         0.00000e+00, 0.00000e+00]]),
 array([[0.19    , 0.09679 , 0.137413, ..., 0.426371, 0.147589, 0.      ],
        [0.18    , 0.054364, 0.118182, ..., 0.41906 , 0.12514 , 0.      ],
        [0.18    , 0.114051, 0.126399, ..., 0.410116, 0.124626, 0.      ],
        ...,
        [0.35    , 0.211002, 0.264161, ..., 0.697398, 0.241205, 0.      ],
        [0.34    , 0.222617, 0.257343, ..., 0.69388 , 0.2953

## Baseline: Direct thresholding sensor observations

In [7]:
traindist = multivariate_normal(mean=np.mean(traindata,axis=0), cov=np.cov(traindata,rowvar=False))

In [ ]:
# For SKAB Data
PFA = 0.05
y_true = []
y_pred = []
for i in range(len(testdata)):
    y_true += list(testlabel[i])
    y_pred += list((traindist.logcdf(testdata[i]) < np.log(PFA/2)).astype(int) | (traindist.logcdf(testdata[i]) > np.log(1-PFA/2)).astype(int))
get_performance(y_true, y_pred)

In [49]:
# For SKAB Data, independent version
PFA = 0.05
y_true = np.array([])
y_pred = np.array([])
traindist = []
for j in range(traindata.shape[1]):
    traindist.append(norm(loc=np.mean(traindata[:,j]), scale=np.sqrt(np.var(traindata[:,j]))))
for m in range(len(testdata)):
    y_true = np.append(y_true, testlabel[m])
    y_pred_j = 0
    for j in range(testdata[m].shape[1]):
        y_pred_j += traindist[j].logcdf(testdata[m][:,j])
    y_pred = np.append(y_pred, (y_pred_j < np.log(PFA/2)).astype(int) | (y_pred_j > np.log(1-PFA/2)).astype(int))
get_performance(y_true, y_pred)

Test Accuracy: 0.35267892896233216
Test Precision: 0.3526624367520681
Test Recall: 0.9948644362208292
Test F1 Score: 0.5207336838360279


In [45]:
# For SMD Data
PFA = 0.05
y_true = np.array([])
y_pred = np.array([])
for m in range(len(traindata)):
    traindist = []
    for j in range(traindata[m].shape[1]):
        traindist.append(norm(loc=np.mean(traindata[m][:,j]), scale=1e-12+np.sqrt(np.var(traindata[m][:,j]))))
    y_true = np.append(y_true, testlabel[m])
    y_pred_j = 0
    for j in range(traindata[m].shape[1]):
        y_pred_j += traindist[j].logcdf(testdata[m][:,j])
    y_pred = np.append(y_pred, (y_pred_j < np.log(PFA/2)).astype(int) | (y_pred_j > np.log(1-PFA/2)).astype(int))
get_performance(y_true, y_pred)

Test Accuracy: 0.041562914655148074
Test Precision: 0.041562914655148074
Test Recall: 1.0
Test F1 Score: 0.07980874524302581


## Classical: Fitting a ARIMA model for each feature

In [27]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler().fit(traindata)
# traindata = ss.transform(traindata)
# mod = VARMAX(traindata, order=(2,2), trend='c')
# res = mod.fit(maxiter=1000, disp=False)
# print(res.summary())

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                          Statespace Model Results                                          
Dep. Variable:     ['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8']   No. Observations:                 9401
Model:                                                   VARMA(2,2)   Log Likelihood              -11135.069
                                                        + intercept   AIC                          22870.139
Date:                                              Thu, 28 Apr 2022   BIC                          25014.710
Time:                                                      23:40:58   HQIC                         23598.297
Sample:                                                           0                                         
                                                             - 9401                                         
Covariance Type:                                                opg                                         
Ljung-Box (L1) (Q):

In [46]:
# obspred = res.forecast(steps=traindata.shape[0])
# pred = np.sum((traindata-obspred)**2, axis=1)
# THR = np.sort(pred)[int((1-PFA)*len(pred))]

In [48]:
# y_true = []
# y_pred = []
# for i in np.random.choice(len(testdata),size=10):
#     y_true += list(testlabel[i])
#     y_pred += list(np.sum((ss.transform(testdata[i])-res.forecast(steps=testdata[i].shape[0]))**2, axis=1) > THR)
# get_performance(y_true, y_pred)

Test Precision: 0.36220994475138124
Test Recall: 0.8428902031370532
Test F1 Score: 0.5066852152407451


In [52]:
# For SKAB
pca = PCA(whiten=True).fit(traindata)
traindata = pca.transform(traindata)

In [53]:
from statsmodels.tsa.stattools import adfuller
order = []
for j in range(traindata.shape[1]):
    pval = 1
    x = traindata[:,j]
    d = -1
    while pval > 0.05:
        d += 1
        res = adfuller(x)
        pval = res[1]
        x = np.diff(x)
    print("Feature", j, "has order", d)
    order.append(d)

Feature 0 has order 0
Feature 1 has order 0
Feature 2 has order 1
Feature 3 has order 0
Feature 4 has order 0
Feature 5 has order 0
Feature 6 has order 0
Feature 7 has order 0


In [85]:
traindist = []
for j in range(traindata.shape[1]):
    traindist.append(ARIMA(traindata[:,j], order=(3,order[j],3)).fit())
    print(traindist[j].summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 9401
Model:                 ARIMA(3, 0, 3)   Log Likelihood              -13337.389
Date:                Fri, 06 May 2022   AIC                          26690.778
Time:                        14:35:21   BIC                          26747.966
Sample:                             0   HQIC                         26710.195
                               - 9401                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.804e-06      0.013     -0.001      1.000      -0.026       0.026
ar.L1         -0.0735      0.932     -0.079      0.937      -1.901       1.754
ar.L2          0.4590      0.440      1.044      0.2

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 9401
Model:                 ARIMA(3, 0, 3)   Log Likelihood               -6632.651
Date:                Fri, 06 May 2022   AIC                          13281.302
Time:                        14:35:28   BIC                          13338.490
Sample:                             0   HQIC                         13300.719
                               - 9401                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0082      0.113      0.072      0.942      -0.213       0.229
ar.L1          1.3210      0.102     12.909      0.000       1.120       1.522
ar.L2         -0.1966      0.092     -2.143      0.0

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 9401
Model:                 ARIMA(3, 0, 3)   Log Likelihood              -13262.093
Date:                Fri, 06 May 2022   AIC                          26540.186
Time:                        14:35:31   BIC                          26597.374
Sample:                             0   HQIC                         26559.603
                               - 9401                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0019      0.022      0.087      0.931      -0.041       0.045
ar.L1          0.1432      0.092      1.554      0.120      -0.037       0.324
ar.L2          0.9893      0.013     75.290      0.0

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 9401
Model:                 ARIMA(3, 0, 3)   Log Likelihood               -5931.753
Date:                Fri, 06 May 2022   AIC                          11879.506
Time:                        14:35:37   BIC                          11936.694
Sample:                             0   HQIC                         11898.923
                               - 9401                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1524      0.225      0.678      0.498      -0.288       0.593
ar.L1          0.9480      0.015     64.560      0.000       0.919       0.977
ar.L2          0.2889      0.017     17.364      0.0

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [92]:
y_true = np.array([])
y_pred = np.array([])
for m in range(len(testdata)):
    y_true = np.append(y_true, testlabel[m]).ravel()
    testdata_tf = pca.transform(testdata[m])
    y_pred_dist = np.zeros(testdata[m].shape)
    sigma2_sum = 0
    for j in range(testdata[m].shape[1]):
        y_pred_dist[:,j] = (traindist[j].forecast(steps=testdata_tf.shape[0]) - testdata_tf[:,j])**2
        sigma2_sum += traindist[j].params[-1]
    y_pred = np.append(y_pred, np.sum(y_pred_dist,axis=1)>sigma2_sum).ravel()
get_performance(y_true, y_pred)

Test Accuracy: 0.3562561734162685
Test Precision: 0.3538535982150058
Test Recall: 0.9941092062533041
Test F1 Score: 0.5219270420301348


In [97]:
# For SMD
y_true = np.array([])
y_pred = np.array([])
for m in range(len(traindata)):
    pca = PCA(whiten=True).fit(traindata[m])
    traindata_tr = pca.transform(traindata[m])
    order = []
    for j in range(traindata_tr.shape[1]):
        pval = 1
        x = traindata_tr[:,j]
        d = -1
        while pval > 0.05:
            d += 1
            res = adfuller(x)
            pval = res[1]
            x = np.diff(x)
        order.append(d)
    traindist = []
    for j in range(traindata_tr.shape[1]):
        traindist.append(ARIMA(traindata_tr[:,j], order=(3,order[j],3)).fit())
#         print(traindist[j].summary())
    y_true = np.append(y_true, testlabel[m]).ravel()
    testdata_tf = pca.transform(testdata[m])
    y_pred_dist = np.zeros(testdata[m].shape)
    sigma2_sum = 0
    for j in range(testdata[m].shape[1]):
        y_pred_dist[:,j] = (traindist[j].forecast(steps=testdata_tf.shape[0]) - testdata_tf[:,j])**2
        sigma2_sum += traindist[j].params[-1]
    y_pred = np.append(y_pred, np.sum(y_pred_dist,axis=1)>sigma2_sum).ravel()
    get_performance(y_true, y_pred)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:924: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:924: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/homebrew/Caskroom/min

Test Accuracy: 0.24508563232936809
Test Precision: 0.0054468652734548685
Test Recall: 0.98
Test F1 Score: 0.010833517576829536


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:924: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Caskroo

KeyboardInterrupt: 

## Classical ML: Isolation Forest

In [32]:
# For SKAB
clf = IsolationForest().fit(traindata)
y_true = np.array([])
y_pred = np.array([])
for m in range(len(testdata)):
    y_true = np.append(y_true, testlabel[m]).ravel()
    y_pred = np.append(y_pred, (clf.predict(testdata[m])<0).astype(int)).ravel()
get_performance(y_true, y_pred)

Test Accuracy: 0.37392882885287915
Test Precision: 0.3579240294977042
Test Recall: 0.9713767842307983
Test F1 Score: 0.5231006995282251


In [34]:
# For SMD
y_true = np.array([])
y_pred = np.array([])
for m in range(len(traindata)):
    clf = IsolationForest().fit(traindata[m])
    y_true = np.append(y_true, testlabel[m]).ravel()
    y_pred = np.append(y_pred, (clf.predict(testdata[m])<0).astype(int)).ravel()
get_performance(y_true, y_pred)

Test Accuracy: 0.6904068208125124
Test Precision: 0.08022426005677245
Test Recall: 0.6162206221980709
Test F1 Score: 0.14196627674973594
